In [ ]:
# !pip install -r requirements.txt -q
from src.training_utils import *

## **MemSum + Pegasus Encoder**

In [4]:
!python src/src/MemSum_Full/train.py -wandb_logger True -pegasus_mode True -training_corpus_file_name src/data/PubMed/train_PUBMED_labelled.jsonl -validation_corpus_file_name src/data/PubMed/val_PUBMED.jsonl -model_folder src/model/MemSum_Full/PubMed/run0/ -log_folder src/log/MemSum_Full/PubMed/run0/ -vocabulary_file_name src/model/glove/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name src/model/glove/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 150 -num_of_epochs 10 -save_every 1000 -n_device 1 -batch_size_per_device 1 -max_extracted_sentences_per_document 7 -moving_average_decay 0.999 -p_stop_thres 0.6




wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in /home/peppe/Desktop/Università/Projects/SAESum-Simultaneous_Abstractive_and_Extractive_Summarization/wandb/run-20230630_212348-bf05iaw3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run chocolate-snowball-17
wandb: ⭐️ View project at https://wandb.ai/deepl_wizards/SAESUM-Abstractive_Extractive_Summarization
wandb: 🚀 View run at https://wandb.ai/deepl_wizards/SAESUM-Abstractive_Extractive_Summarization/runs/bf05iaw3
True
116937it [00:05, 20372.08it/s]
6633it [00:00, 28443.71it/s]
PegasusTokenizerFast(name_or_path='google/pegasus-x-base', vocab_size=96103, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_

## **Evaluation**

In [ ]:
from src.summarizers import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

In [ ]:
rouge_cal = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

memsum_custom_data = MemSum("src/model/MemSum_Full/custom_data/200dim/run3/model_batch_1000_pegs.pt",
                            "src/model/glove/vocabulary_200dim.pkl",
                            gpu=True,  max_doc_len=500, pegasus_mode=True, embed_dim=768)

In [ ]:
test_corpus_custom_data = [ json.loads(line) for line in open("src/data/custom_data/test_CUSTOM_raw.jsonl")]

In [ ]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]

        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )

    return np.asarray(scores).mean(axis = 0)

In [ ]:
evaluate( memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal)